<a href="https://colab.research.google.com/github/last-war/HW_DataScience/blob/Homework-09/HW_09.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/https-deeplearning-ai/tensorflow-1-public/blob/master/C1/W3/ungraded_labs/C1_W3_Lab_1_improving_accuracy_using_convolutions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


В якості домашнього завдання вам пропонується створити нейронну мережу за допомогою механізмів Keras, яка буде класифікувати товари із датасету fasion_mnist.

Вам належить запропонувати свою власну архітектуру мережі. Точність найнаївнішої, але адекватної нейромережі становить приблизно 91%. Точність вашої моделі повинна бути не нижчою за цей показник. Щоб досягти таких значень вам знадобиться поекспериментувати з гіперпараметрами мережі:

кількість шарів;

кількість нейронів;

функції активації;

кількість епох;

розмір батчу;

вибір оптимізатора;

різні техніки регуляризації і т.д.

Використайте вивчені техніки виявлення проблем навчання нейронної мережі, і потім поекспериментуйте.

Рішення оформіть у вигляді окремого ноутбука.

In [ ]:
import tensorflow as tf

# завантаження Fashion MNIST
fmnist = tf.keras.datasets.fashion_mnist
(training_images, training_labels), (test_images, test_labels) = fmnist.load_data()

# нормалізація
training_images = training_images / 255.0
test_images = test_images / 255.0

In [ ]:
#перевірка сету

import matplotlib.pyplot as plt
import random

for itr in range(2):
  id_pic = random.randint(0, test_images.shape[1])
  plt.imshow(test_images[id_pic].reshape(28, 28), cmap="gray")
  plt.show()

Для припинення навчання

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    '''
    Halts the training after reaching 60 percent accuracy

    Args:
      epoch (integer) - index of epoch (required but unused in the function definition below)
      logs (dict) - metric results from the training epoch
    '''

    # Check accuracy
    if(logs.get('accuracy') > 0.98):
      print("\nGet is accuracy than 0.98 so cancelling training!")
      self.model.stop_training = True

    if(logs.get('loss') < 0.1):
      print("\nLoss is lower than 0.1 so cancelling training!")
      self.model.stop_training = True

# Instantiate class
callbacks = myCallback()

**Optimaze**

tf.keras.optimizers.experimental.Adadelta(
    learning_rate=0.001,
    rho=0.95,
    epsilon=1e-07,
    weight_decay=None,
    clipnorm=None,
    clipvalue=None,
    global_clipnorm=None,
    use_ema=False,
    ema_momentum=0.99,
    ema_overwrite_frequency=None,
    jit_compile=True,
    name='Adadelta',
    **kwargs
)

tf.keras.optimizers.experimental.RMSprop(
    learning_rate=0.001,
    rho=0.9,
    momentum=0.0,
    epsilon=1e-07,
    centered=False,
    weight_decay=None,
    clipnorm=None,
    clipvalue=None,
    global_clipnorm=None,
    use_ema=False,
    ema_momentum=0.99,
    ema_overwrite_frequency=100,
    jit_compile=True,
    name='RMSprop',
    **kwargs
)

tf.keras.optimizers.experimental.SGD(
    learning_rate=0.01,
    momentum=0.0,
    nesterov=False,
    weight_decay=None,
    clipnorm=None,
    clipvalue=None,
    global_clipnorm=None,
    use_ema=False,
    ema_momentum=0.99,
    ema_overwrite_frequency=None,
    jit_compile=True,
    name='SGD',
    **kwargs
)

tf.keras.optimizers.Adam(
    learning_rate=0.001,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07,
    amsgrad=False,
    weight_decay=None,
    clipnorm=None,
    clipvalue=None,
    global_clipnorm=None,
    use_ema=False,
    ema_momentum=0.99,
    ema_overwrite_frequency=None,
    jit_compile=True,
    name='Adam',
    **kwargs
)

Для всіх:

батчі [10, 100, 500, 1000]

епох [5, 10, 20, 50]

4 оптимізаторів (вище)

[Adadelta, RMSprop, SGD, Adam]

func activ:

relu(...) -  перевірити

sigmoid(...) - перевірити

softmax(...) - для фінального шару

1) проста модель без згортання
нейронів 784

2) модель +1 згортання+зтискання

3) + drops

In [ ]:

l_optims = ['Adadelta', 'RMSprop', 'SGD', 'Adam']

l_batchs = [100, 500, 1000]

#по першій моделі і першому оптимізатору: прийняв рішення відкинути порції менше 100. Та зменшити максимальну кількість епох до 30
#батч по 10 лише збільшує час навчання, без значного ефекту, як і велика кількість єпох
#l_batchs = [10, 100, 500, 1000]
#l_epochs = [5, 10, 20, 50]

l_epochs = [10, 20, 30]




In [ ]:
# model_1
# прохід для пошуку параметрів

for cur_opt in l_optims:
  for cur_epoch in l_epochs:
    for cur_batch in l_batchs:
      model = tf.keras.models.Sequential([
              tf.keras.layers.Flatten(input_shape=(28, 28)),
              tf.keras.layers.Dense(512, activation=tf.nn.relu),
              tf.keras.layers.Dense(10, activation=tf.nn.softmax)
                                          ])
      model.compile(optimizer=cur_opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

      # Train the model
      model.fit(training_images, training_labels, batch_size=cur_batch, epochs=cur_epoch, callbacks=[callbacks])

      # Evaluate on the test set

      test_loss = model.evaluate(test_images, test_labels)
      print(f'\nMODEL 1. optimizer={cur_opt} batch_size={cur_batch} epochs={cur_epoch}:')


Adadelta максимально отримано на batch_size=100 epochs=50: 80%

RMSprop максимально отримано на
batch_size=100 epochs=20: 89%
batch_size=500 epochs=20: 89%
batch_size=500 epochs=50: 89%
batch_size=1000 epochs=50: 89%

SGD максимально отримано на
batch_size=100 epochs=50: 86%

Adam максимально отримано на
batch_size=100 epochs=10: 89%
batch_size=100 epochs=13/20: 88%
batch_size=500 epochs=20: 89%
batch_size=100 epochs=15/50: 88%
batch_size=500 epochs=19/50: 89%
batch_size=1000 epochs=25/50: 89%


Змінив параметри коллбеку до 0.1 втрат і 98 точність

In [ ]:
# model_1
# контроль на обраних параметрах

model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(512, activation=tf.nn.relu),
        tf.keras.layers.Dense(10, activation=tf.nn.softmax)
                                    ])
model.compile(optimizer='Adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

# Train the model
print(f'\nMODEL TRAINING:')
history = model.fit(training_images, training_labels, batch_size=250, epochs=30, callbacks=[callbacks], validation_data=(test_images, test_labels))


# Plot the results
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()

plt.show()


In [ ]:
# model_2

for cur_opt in l_optims:
  for cur_epoch in l_epochs:
    for cur_batch in l_batchs:
      model = tf.keras.models.Sequential([
              tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(28, 28, 1)),
              tf.keras.layers.MaxPooling2D(2, 2),
              tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
              tf.keras.layers.MaxPooling2D(2,2),
              tf.keras.layers.Flatten(),
              tf.keras.layers.Dense(256, activation=tf.nn.relu),
              tf.keras.layers.Dense(10, activation=tf.nn.softmax)
                                          ])
      model.compile(optimizer=cur_opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

      # Train the model
      model.fit(training_images, training_labels, batch_size=cur_batch, epochs=cur_epoch, callbacks=[callbacks])

      # Evaluate on the test set
      test_loss = model.evaluate(test_images, test_labels)
      print(f'\nMODEL 1. optimizer={cur_opt} batch_size={cur_batch} epochs={cur_epoch}:')




висновки
Швикість роботи збільшено, результати співставні і краще


Adadelta максимально отримано на batch_size=100 epochs=30: 71%
(стало гірше швидкість навчання вища, даний оптимізатор не підходить для таких задач)

RMSprop максимально отримано на batch_size=100-500 epochs=20-30: 91%
(високий результат, але помітна помилка на валідаційних даних)

SGD максимально отримано на batch_size=100 epochs=30: 88%
(результат краще, проте цей оптимізатор не відповідає задачі)

Adam максимально отримано на batch_size=100-500 epochs=20-30: 91%
(ідентично до RMS, навіть на наявних набор при валідації значно зростають втрати. Великий ризик перенавчання)


In [ ]:
# model_2
# контроль на обраних параметрах

model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(28, 28, 1)),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(256, activation=tf.nn.relu),
        tf.keras.layers.Dense(10, activation=tf.nn.softmax)
                                    ])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

# Train the model
print(f'\nMODEL TRAINING:')
history = model.fit(training_images, training_labels, batch_size=250, epochs=25, callbacks=[callbacks], validation_data=(test_images, test_labels))


# Plot the results
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()

plt.show()

In [ ]:
# model_3 add drops

for cur_opt in l_optims:
  for cur_epoch in l_epochs:
    for cur_batch in l_batchs:
      model = tf.keras.models.Sequential([
              tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(28, 28, 1)),
              tf.keras.layers.MaxPooling2D(2, 2),
              tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
              tf.keras.layers.MaxPooling2D(2,2),
              tf.keras.layers.Dropout(0.1),
              tf.keras.layers.Flatten(),
              tf.keras.layers.Dense(256, activation=tf.nn.relu),
              tf.keras.layers.Dense(10, activation=tf.nn.softmax)
                                          ])
      model.compile(optimizer=cur_opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

      # Train the model
      model.fit(training_images, training_labels, batch_size=cur_batch, epochs=cur_epoch, callbacks=[callbacks])

      # Evaluate on the test set
      test_loss = model.evaluate(test_images, test_labels)
      print(f'\nMODEL 3. optimizer={cur_opt} batch_size={cur_batch} epochs={cur_epoch}:')





висновки Швикість роботи співставна, результати певних оптимізаторів стали гірше

Adadelta максимально отримано на batch_size=100 epochs=30: 73% (майже ідентично, даний оптимізатор не підходить для таких задач)

RMSprop максимально отримано на batch_size=500 epochs=30: 92% (високий результат, при тому що тестова вибірка дає 96%)

SGD максимально отримано на batch_size=100 epochs=30: 88% (результат ідентичний, проте цей оптимізатор не відповідає задачі)

Adam максимально отримано на batch_size=500 epochs=30: 92% (найкращий результат на валідаційних даних)


In [ ]:
# model 3 оптимальні параметри
model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(28, 28, 1)),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(256, activation=tf.nn.relu),
        tf.keras.layers.Dense(10, activation=tf.nn.softmax)
                                    ])

# Print the model summary
model.summary()

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
print(f'\nMODEL TRAINING:')
history = model.fit(training_images, training_labels, batch_size=500, epochs=30, callbacks=[callbacks], validation_data=(test_images, test_labels))



# Plot the results
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()

plt.show()


Хочу додатково перевірити кілька вариантів без коллбеку, але з іншою кількостю нейронів
відносно великою і відносно малою

In [ ]:
# model 3 але мало нейронів
model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(28, 28, 1)),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation=tf.nn.relu),
        tf.keras.layers.Dense(10, activation=tf.nn.softmax)
                                    ])

# Print the model summary
model.summary()

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
print(f'\nMODEL TRAINING:')
history = model.fit(training_images, training_labels, batch_size=500, epochs=50, validation_data=(test_images, test_labels))



# Plot the results
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy 64-10')
plt.legend(loc=0)
plt.figure()

plt.show()


In [ ]:
# model 3 але багато нейронів +доп рівень
model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(28, 28, 1)),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(512, activation=tf.nn.relu),
        tf.keras.layers.Dense(256, activation=tf.nn.relu),
        tf.keras.layers.Dense(10, activation=tf.nn.softmax)
                                    ])

# Print the model summary
model.summary()

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
print(f'\nMODEL TRAINING:')
history = model.fit(training_images, training_labels, batch_size=500, epochs=50, validation_data=(test_images, test_labels))



# Plot the results
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy 512-256-10')
plt.legend(loc=0)
plt.figure()

plt.show()


Збільшена кількість шарів та нейронів прямий шлях до перенавчання (якщо мова про цей сет з розмірностю 28*28)
варіація з 64 нейронами вже з 10 епох дала результати понад 90% на валідаційному наборі
Оптимізатор, для завдань розпізнання - Адам.
Функція софтмакс - так як класів понад 2
щодо батчу - на мою думку 1/500 від виборки оптимальний варіант. Це дає достатню швидкість при цього вибірки вистачає для визначення коєф.
Кількість епох варто обмежувати на рівні 30-50, але краще використовувати перевірку точності та втрат, для припинення навчання.
Щодо перенавчання: на такому малому наборі його не уникнути, але можна зменшити, шляхом як дропу, так і перериванням навчання